In [1]:
@file:DependsOn("com.google.adk:google-adk:0.3.0")
@file:DependsOn("com.google.adk:google-adk-dev:0.2.0")
@file:DependsOn("com.google.genai:google-genai:1.23.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-rx3:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.8.1")

In [2]:
import com.google.adk.agents.LlmAgent
import com.google.adk.agents.SequentialAgent
import com.google.adk.agents.ParallelAgent
import com.google.adk.agents.LoopAgent

import com.google.adk.models.Gemini
import com.google.genai.Client

import com.google.adk.runner.InMemoryRunner
import com.google.adk.sessions.Session
import com.google.adk.sessions.InMemorySessionService
import com.google.adk.memory.InMemoryMemoryService
import com.google.adk.events.Event
import com.google.adk.tools.FunctionTool
import com.google.adk.tools.GoogleSearchTool

import com.google.genai.types.Content
import com.google.genai.types.Part

import java.util.UUID
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.reactive.asFlow

import io.reactivex.rxjava3.core.Flowable;

In [3]:
val apiKey = System.getenv("GOOGLE_API_KEY")
if (apiKey == null) {
    throw IllegalStateException("Please set the GOOGLE_API_KEY environment variable.")
}

In [4]:
val geminiClient = Client.builder().apiKey(apiKey).build()

In [5]:
val geminiModel = Gemini(
    "gemini-2.5-flash-lite", // Your model name
    geminiClient
)

---

## 🤖 Section 2: Your first AI Agent with ADK

### 🤔 2.1 What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

`Prompt -> LLM -> Text`

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

In [6]:
val rootAgent = LlmAgent.builder()
    .name("helpful_assistant")
    .model(geminiModel)
    .description("A simple agent that can answer general questions")
    .instruction("""You are a helpful assistant. Use Google Search for current info or if unsure""")
    .tools(GoogleSearchTool.INSTANCE)
    .build()

println("✅ Root agent defined.")

✅ Root agent defined.


In [7]:
var runner: InMemoryRunner = InMemoryRunner(rootAgent, "Assistant Flow")

In [8]:
var prompt = "What is Agent Development Kit from Google? What languages is the SDK available in"

var userMsg: Content? = Content.fromParts(Part.fromText(prompt))

In [9]:
var session: Session? = runner
    .sessionService()
    .createSession("Assistant Flow", "user")
    .blockingGet()

In [15]:
println("Agent response ->")
runBlocking {
    var events = runner.runAsync("user", session.id(), userMsg)
        .asFlow()
        .toList()
    events.forEach {
        event -> println(event.stringifyContent())
    }
}

Agent response ->
Kraftwerk has two upcoming concerts in Berlin in December 2025. They are scheduled to perform on **December 9 and December 10, 2025**, both at the Uber Eats Music Hall. These performances are part of their "Multimedia Tour 2025."


In [11]:
prompt = "What did I just ask you about"
userMsg = Content.fromParts(Part.fromText(prompt))
events = runner.runAsync("user", session.id(), userMsg)
println("Agent response ->")
events.blockingForEach { event -> println(event.stringifyContent()) }

Agent response ->
You asked me about the Agent Development Kit (ADK) from Google and the programming languages in which the SDK is available.


In [12]:
prompt = "Can you tell me when is the next Kraftwerk concert in Berlin?"
userMsg = Content.fromParts(Part.fromText(prompt))
events = runner.runAsync("user", session.id(), userMsg)
println("Agent response ->")
events.blockingForEach { event -> println(event.stringifyContent()) }

Agent response ->
Kraftwerk has two upcoming concerts in Berlin on **December 9 and 10, 2025**, both at the Uber Eats Music Hall. Tickets for these shows are available, with some already listed as "few tickets left" or "about to be sold out."
